# (some) Related Work

- Solving Inverse PDE Problems using Grid-Free Monte Carlo Estimators
- Monte Carlo Geometry Processing: A Grid-Free Approach to PDE-Based Methods on Volumetric Domains
- Grid-Free Monte Carlo for PDEs with Spatially Varying Coefficients
- Sparsified Randomization Algorithms for large systems of linear equations and a new version of the Random Walk on Boundary method


# Abstract
todo

# Interesting Examples  
We haven't addressed which integral equations produce well behaving estimators. Following examples show some bad behavior.

## Example 1

Bad example of turning a non recursive integral equation into a recursive one:

$$
\begin{align*}
e^{t} &= 1 + \int_{0}^{t} y(s) ds \Leftrightarrow\\
y(t)  &= 1 + \int_{0}^{t} y(s) ds - e^{t} + y(t)
\end{align*}
$$

then use russian roulette to remove branching. We think that it is impossible to turn this last recursive integral equation into a finite variance estimator.


In [19]:
from random import random, randint
from math import exp


def Y(t, q):  # t<1
    k = randint(1, 3)
    if k == 1:
        return q*(1 - exp(t))+1+t
    if k == 2:
        return q*(1 - exp(t)) + 3*(Y(t, q)-1-t)
    return q*(1 - exp(t) + 3*t*Y(random()*t, q))


sol = 0
nsim = 10**1
t = 0.5  # <1

for _ in range(nsim):
    sol += Y(t, -0.1)/nsim

s = exp(t)  # analytic solution
percentage_error = (sol - s)/s

print(f"sol({t}) is approx = {sol}")
print(f"%error = {percentage_error}")


sol(0.5) is approx = -0.7605162916145758
%error = -1.4612764480751943


## Example 2

Alternative approach for ODE example in period 2

$$
\begin{aligned}
y''&=y  \Leftarrow \\
y'(t)&=y'(1)-\int_{t}^{1} y(s)ds \Leftarrow\\
y(t)&=y(0)+t y'(1)-\int_0^t \int_s^1 y(l) d l d s
\end{aligned}
$$

with $y(0)=1, y'(1) = e$ such that the solution is $y(t) = e^{t}$. The implementation uses Russian Roulette:


In [20]:
from random import random
from math import exp


def Y(t):  # 0<t<1
    S = random()*t
    L = 1-(1 - S)*random()
    return 1 + t*exp(1) - 2 * t * (1-S) * Y(L) if random() < 1/2 else 1 + t*exp(1)


sol = 0
nsim = 10**4
t = 0.5  # <1

for _ in range(nsim):
    sol += Y(t)/nsim

s = exp(t)  # analytic solution
percentage_error = (sol - s)/s

print(f"sol({t}) is approx = {sol}")
print(f"%error = {percentage_error}")


sol(0.5) is approx = 1.666418107956915
%error = 0.010733674376185961


If you now increase the domain to $[0,2]$ the variance explodes (we conjecture that the variance abides a ODE and exists):

In [21]:
from random import random,randint
from math import exp

def Y(t): # 0<t<2
    S = random()*t
    L = 2-(2 - S)*random()
    if t<1:
        return  1 + t*exp(2) - (2-S)* Y(L) if random()<t else  1 + t*exp(2)  
    return  1 + t*exp(2) - t*(2-S)* Y(L) 

sol = 0
nsim = 10**2
t = 0.5 # <2

for _ in range(nsim):
    sol += Y(t)/nsim

s = exp(t) #analytic solution
percentage_error = (sol - s)/s

print(f"sol({t}) is approx = {sol}")
print(f"%error = {percentage_error}")

sol(0.5) is approx = 217745069.3372903
%error = 132069059.55431978


## Mini Conclusion
You have to be careful to get good properties when using RMC. Even if the variance exist, it may be unreasonably high. There are multiple ways to get the variance down the reasonable levels importance sampling, using physical interpretations, next flight (see later),...

# Next Flight
The idea behind next flight or something more general is recursion in recursion like the induction in induction proofs we love. By recursion in recursion we mean using a recursion for solving an other recursion step, $1$ slow recursion, $1$ fast recursion or $1$ outer recursion, $1$ inner recursion. <br>

In Grid-Free Monte Carlo for PDEs with Spatially Varying Coefficients they implement a next flight for walk on spheres and they talk about small walks = fast recursion in a big walk = slow recursion. <br>

Lets look at following example:
$$
y'=y, y(0)=1.
$$
with solution $y(t)=e^{t}$ this can be turned in following integral equation:
$$
y(t)= y(T) +  \int_{T}^{t}y(s)ds   
$$ 
in the slow recursion we estimate $y(T)$ with a call to the fast recursion using that estimate we estimate the integral with RMC leading to a fast recursion and base case is the initial condition. (maybe that is still confusing ...) <br>

Implementation of this looks like:

In [99]:
from random import random
from math import exp


def Y_fast(t, T, yT, DT):
    S = T + random()*(t-T)  # \sim Uniform(T,t)
    return yT + DT*Y_fast(S, T, yT, DT) if random() < (t-T)/DT else yT


def Y_slow(T, DT: "step size slow recursion"):
    TT = T-DT if T-DT > 0 else 0  # TT is where we are recursing to
    return Y_fast(T, TT, Y_slow(TT, DT), DT) if T > 0 else 1


# the convergence speeds is probably O(DT/sqrt(nsim))
s = 0
t = 30
DT = 0.25
nsim = 10**2

for _ in range(nsim):
    s += Y_slow(t, DT)/nsim

sol = exp(t)
err = (s-sol)/sol
print(f"res = {s}, ",f"%error= {err}")


res = 10522854145087.832,  %error= -0.015310983541710706


We look into high order methods in $DT$ later.

# Coupled Recursion